In [51]:
!pip install tensorflow-addons

In [52]:
!wget http://nlp.stanford.edu/data/glove.6B.zip


--2020-06-25 17:02:08--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-06-25 17:02:09--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-06-25 17:02:09--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [53]:
import numpy as np
import os
import numpy as np
import matplotlib.pyplot as plt


from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply , Embedding
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

import tensorflow as tf
import tensorflow_addons as tfa
print(tf.__version__)
from sklearn.model_selection import train_test_split
import os
import io
import numpy as np
import re


2.2.0


In [54]:
!unzip glove.6B.zip -d gloves

Archive:  glove.6B.zip
replace gloves/glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [55]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

In [117]:
from string import digits
from sklearn.model_selection import train_test_split
import os
import io
import numpy as np
import re
import unicodedata
import urllib3
import shutil
import zipfile
import itertools
def preprocess_sentence(sentence):
    
    num_digits= str.maketrans('','', digits)
    
    sentence= sentence.lower()
    sentence= re.sub(" +", " ", sentence)
    sentence= re.sub("'", '', sentence)
    sentence= sentence.translate(num_digits)
    sentence= re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = sentence.rstrip().strip()
    sentence = "<start> " + sentence + " <end>"
    
    return sentence


In [57]:
words, word_to_vec_map = read_glove_vecs('/content/gloves/glove.6B.50d.txt')
print(np.shape(word_to_vec_map["father"]))
input_sentences=[]

filepath = '/content/newur'
with open(filepath) as fp:
    line = fp.readline()
    cnt = 1
    while cnt <3000 :
        line = fp.readline()
        if cnt >15:
            input_sentences.append(line)
        cnt+=1
print(len(input_sentences))

def wordcleaner(word):
    temp=""
    for char in word:
#         print(char)
        if char!= "," and char!= "!" and char!= "?" and char!= "." and char!= "'" and char!= "-":
#         if char!=',':
            temp+=char
    return temp
wordcleaner("i'm")

(50,)
2984


'im'

In [119]:
input_sent_cleaned=[]
print(input_sentences[0])
for l in input_sentences:
#     print(l)
    temp=[]
#     for word in l.lower().split():
# #         print(word)
#         word = wordcleaner(word)
# #         print(word)
# #         if word!="":
# #         w = word_to_vec_map.get(word)
#         temp.append(word)
# #         print(np.shape(temp))
    temp = preprocess_sentence(l)
    input_sent_cleaned.append(temp)
    
print(input_sent_cleaned[4])

That would hurt her and I don't ever want to do that.

<start> around the hill a warriors destroying our men . <end>


In [194]:
vocab_size = 10000
embedding_dim = 100
max_length = 25
trunc_token='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(oov_token=oov_tok , filters = "")
tokenizer.fit_on_texts(input_sent_cleaned)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(input_sent_cleaned)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_token,padding = 'post')
print(input_sent_cleaned[8])
print(padded[8])
print(np.shape(padded))

index_word = {}
for word , index in word_index.items():
    index_word[index] = word

<start> in the time of ancient gods ,  warlords and kings a land in turmoil cried out for a hero . <end>
[   2   17    8  106   13  785  240    6 1268   14  452    9 1269   17
 1270 1271   80   22    9  786    4    3    0    0    0]
(2984, 25)


In [195]:
len(word_index)

2916

In [197]:
batch_size = 1000
max_len = 25
context = np.zeros((1000,max_len))
response = np.zeros((1000,max_len))


In [248]:
index=0
for idx in range(0,2000,2):
    context[index,:] = padded[idx]
    response[index,:] = padded[idx+1]
    index +=1
    
print(np.shape(context))
print(context[1])
print(response[0])
# response = list(response.swapaxes(0,1))
# context

(1000, 25)
[  2.  10. 187.   5.   6. 121.   4.   3.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
[  2.  93.  20. 388.   4.   3.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [199]:
import os
embeddings_index = {}
f = open( '/content/gloves/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector 
        
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            50,
                            weights=[embedding_matrix],
                            input_length=20,
                            trainable=False)

print("Total Samples : ", len(context))

Found 400000 word vectors.
Total Samples :  1000


In [225]:
BATCH_SIZE = 100
BUFFER_SIZE = len(context)
steps_per_epoch = BUFFER_SIZE//BATCH_SIZE
embedding_dims = 50
rnn_units = 1000
dense_units = 1000
Dtype = tf.float32  
print(BUFFER_SIZE)

1000


In [226]:
print(np.shape(context))
np.shape(response)

(1000, 25)


(1000, 25)

In [227]:
dataset = tf.data.Dataset.from_tensor_slices((context,response)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
example_X, example_Y = next(iter(dataset))
print(example_X.shape) 
print(example_Y.shape)

(100, 25)
(100, 25)


In [228]:
Tx=25
Ty=25
vocab_size = 1+len(word_index)

In [229]:
#ENCODER
class EncoderNetwork(tf.keras.Model):
    def __init__(self,input_vocab_size,embedding_dims, rnn_units, embeddings_matrix ):
        super().__init__()
        self.encoder_embedding = tf.keras.layers.Embedding(input_dim=input_vocab_size,weights = [embeddings_matrix],
                                                           output_dim=embedding_dims)
        self.encoder_rnnlayer = tf.keras.layers.LSTM(rnn_units,return_sequences=True, 
                                                     return_state=True )
    

In [230]:
#DECODER
class DecoderNetwork(tf.keras.Model):
    def __init__(self,output_vocab_size, embedding_dims, rnn_units , embeddings_matrix):
        super().__init__()
        self.decoder_embedding = tf.keras.layers.Embedding(input_dim = output_vocab_size,
                                                       output_dim = embedding_dims,
                                                       weights = [embeddings_matrix], trainable = False) 
        self.dense_layer = tf.keras.layers.Dense(output_vocab_size)
        self.decoder_rnncell = tf.keras.layers.LSTMCell(rnn_units)
        # Sampler
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        # Create attention mechanism with memory = None
        self.attention_mechanism = self.build_attention_mechanism(dense_units,None,BATCH_SIZE*[Tx])
        self.rnn_cell =  self.build_rnn_cell(BATCH_SIZE)
        self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler= self.sampler,
                                                output_layer=self.dense_layer)

    def build_attention_mechanism(self, units,memory, memory_sequence_length):
        return tfa.seq2seq.LuongAttention(units, memory = memory, 
                                          memory_sequence_length=memory_sequence_length)
        #return tfa.seq2seq.BahdanauAttention(units, memory = memory, memory_sequence_length=memory_sequence_length)

    # wrap decodernn cell  
    def build_rnn_cell(self, batch_size ):
        rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnncell, self.attention_mechanism,
                                                attention_layer_size=dense_units)
        return rnn_cell
    
    def build_decoder_initial_state(self, batch_size, encoder_state,Dtype):
        decoder_initial_state = self.rnn_cell.get_initial_state(batch_size = batch_size, 
                                                                dtype = Dtype)
        decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state) 
        return decoder_initial_state



In [231]:
encoderNetwork = EncoderNetwork(len(word_index)+1,embedding_dims, rnn_units,embedding_matrix)
decoderNetwork = DecoderNetwork(len(word_index)+1,embedding_dims, rnn_units , embedding_matrix)

In [232]:
optimizer = tf.keras.optimizers.Adam()

In [233]:
def loss_function(y_pred, y):
   
    #shape of y [batch_size, ty]
    #shape of y_pred [batch_size, Ty, output_vocab_size] 
    sparsecategoricalcrossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                                  reduction='none')
    loss = sparsecategoricalcrossentropy(y_true=y, y_pred=y_pred)
    #skip loss calculation for padding sequences i.e. y = 0 
    #[ <start>,How, are, you, today, 0, 0, 0, 0 ....<end>]
    #[ 1, 234, 3, 423, 3344, 0, 0 ,0 ,0, 2 ]
    # y is a tensor of [batch_size,Ty] . Create a mask when [y=0]
    # mask the loss when padding sequence appears in the output sequence
    mask = tf.logical_not(tf.math.equal(y,0))   #output 0 for y=0 else output 1
    mask = tf.cast(mask, dtype=loss.dtype)
    loss = mask* loss
    loss = tf.reduce_mean(loss)
    return loss

In [234]:
decoderNetwork.attention_mechanism.memory_initialized

False

In [235]:
def train_step(input_batch, output_batch,encoder_initial_cell_state):
    #initialize loss = 0
    loss = 0
    with tf.GradientTape() as tape:
        encoder_emb_inp = encoderNetwork.encoder_embedding(input_batch)
        a, a_tx, c_tx = encoderNetwork.encoder_rnnlayer(encoder_emb_inp, 
                                                        initial_state =encoder_initial_cell_state)

        #[last step activations,last memory_state] of encoder passed as input to decoder Network
        
         
        # Prepare correct Decoder input & output sequence data
        decoder_input = output_batch[:,:-1] # ignore <end>
        #compare logits with timestepped +1 version of decoder_input
        decoder_output = output_batch[:,1:] #ignore <start>


        # Decoder Embeddings
        decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)

        #Setting up decoder memory from encoder output and Zero State for AttentionWrapperState
        decoderNetwork.attention_mechanism.setup_memory(a)
        decoder_initial_state = decoderNetwork.build_decoder_initial_state(BATCH_SIZE,
                                                                           encoder_state=[a_tx, c_tx],
                                                                           Dtype=tf.float32)
        
        #BasicDecoderOutput        
        outputs, _, _ = decoderNetwork.decoder(decoder_emb_inp,initial_state=decoder_initial_state,
                                               sequence_length=BATCH_SIZE*[Ty-1])

        logits = outputs.rnn_output
        #Calculate loss

        loss = loss_function(logits, decoder_output)

    #Returns the list of all layer variables / weights.
    variables = encoderNetwork.trainable_variables + decoderNetwork.trainable_variables  
    # differentiate loss wrt variables
    gradients = tape.gradient(loss, variables)

    #grads_and_vars – List of(gradient, variable) pairs.
    grads_and_vars = zip(gradients,variables)
    optimizer.apply_gradients(grads_and_vars)
    return loss

In [236]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True )

Mounted at /content/drive


In [237]:

checkpointdir = os.path.join('/content/drive/My Drive/',"checkpoint")
chkpoint_prefix = os.path.join(checkpointdir, "checkpoint")
if not os.path.exists(checkpointdir):
    os.mkdir(checkpointdir)

checkpoint = tf.train.Checkpoint(optimizer = optimizer, encoderNetwork = encoderNetwork, 
                                 decoderNetwork = decoderNetwork)

try:
    status = checkpoint.restore(tf.train.latest_checkpoint(checkpointdir))
    print("Checkpoint found at {}".format(tf.train.latest_checkpoint(checkpointdir)))
except:
    print("No checkpoint found at {}".format(checkpointdir))

Checkpoint found at None


In [238]:
#RNN LSTM hidden and memory state initializer
def initialize_initial_state():
        return [tf.zeros((BATCH_SIZE, rnn_units)), tf.zeros((BATCH_SIZE, rnn_units))]

In [249]:
epochs = 100
for i in range(1, epochs+1):
    
    encoder_initial_cell_state = initialize_initial_state()
    total_loss = 0.0


    for ( batch , (input_batch, output_batch)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(input_batch, output_batch, encoder_initial_cell_state)
        total_loss += batch_loss
        # print(batch)
        if (batch+1)%10 == 0:
            print("total loss: {} epoch {} batch {} ".format(batch_loss.numpy(), i, batch+1))
            checkpoint.save(file_prefix = chkpoint_prefix)

total loss: 1.1358423233032227 epoch 1 batch 10 
total loss: 1.1184232234954834 epoch 2 batch 10 
total loss: 0.9790633320808411 epoch 3 batch 10 
total loss: 1.0546746253967285 epoch 4 batch 10 
total loss: 0.8833094239234924 epoch 5 batch 10 
total loss: 0.9288177490234375 epoch 6 batch 10 
total loss: 0.8195549845695496 epoch 7 batch 10 
total loss: 0.7744458317756653 epoch 8 batch 10 
total loss: 0.7122776508331299 epoch 9 batch 10 
total loss: 0.8810794353485107 epoch 10 batch 10 
total loss: 0.9990411400794983 epoch 11 batch 10 
total loss: 0.7876838445663452 epoch 12 batch 10 
total loss: 0.6820161938667297 epoch 13 batch 10 
total loss: 0.5455398559570312 epoch 14 batch 10 
total loss: 0.5333096385002136 epoch 15 batch 10 
total loss: 0.4550606906414032 epoch 16 batch 10 
total loss: 0.4207104444503784 epoch 17 batch 10 
total loss: 0.36501407623291016 epoch 18 batch 10 
total loss: 0.3066101372241974 epoch 19 batch 10 
total loss: 0.27662667632102966 epoch 20 batch 10 
total l

KeyboardInterrupt: ignored

In [250]:
decoder_embedding_matrix = decoderNetwork.decoder_embedding.variables[0] 
print(decoderNetwork.decoder_embedding.variables[0].shape)

(2917, 50)


In [251]:
beam_width = 3
#use with scope /cpu:0 for inferencing
#restore from latest checkpoint for inferencing
input_raw=" how was your day\n who are you \n do you support him? \n easy way to die \n find a girl for me \n When will this job get over? "
#input_raw="Wow!"  #checking translation on training set record
#def inference(input_raw):
input_lines = input_raw.split("\n")
# We have a transcript file containing English-Hindi pairs
# Preprocess X
input_lines = [preprocess_sentence(line) for line in input_lines]
input_sequences = [[word_index[w] for w in line.split(' ')] for line in input_lines]
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences,
                                                                maxlen=Tx, padding='post')
inp = tf.convert_to_tensor(input_sequences)
#print(inp.shape)
inference_batch_size = input_sequences.shape[0]
encoder_initial_cell_state = [tf.zeros((inference_batch_size, rnn_units)),
                              tf.zeros((inference_batch_size, rnn_units))]
encoder_emb_inp = encoderNetwork.encoder_embedding(inp)
a, a_tx, c_tx = encoderNetwork.encoder_rnnlayer(encoder_emb_inp,
                                                initial_state =encoder_initial_cell_state)

start_tokens = tf.fill([inference_batch_size],word_index['<start>'])
#print(start_tokens)
end_token = word_index['<end>']



decoder_input = tf.expand_dims([word_index['<start>']]* inference_batch_size,1)
decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)


#From official documentation
#NOTE If you are using the BeamSearchDecoder with a cell wrapped in AttentionWrapper, then you must ensure that:

#The encoder output has been tiled to beam_width via tfa.seq2seq.tile_batch (NOT tf.tile).
#The batch_size argument passed to the get_initial_state method of this wrapper is equal to true_batch_size * beam_width.
#The initial state created with get_initial_state above contains a cell_state value containing properly tiled final state from the encoder.
encoder_memory = tfa.seq2seq.tile_batch(a, beam_width)
decoderNetwork.attention_mechanism.setup_memory(encoder_memory)
print("beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] :", encoder_memory.shape)
#set decoder_inital_state which is an AttentionWrapperState considering beam_width
decoder_initial_state = decoderNetwork.rnn_cell.get_initial_state(batch_size = inference_batch_size* beam_width,dtype = Dtype)
encoder_state = tfa.seq2seq.tile_batch([a_tx, c_tx], multiplier=beam_width)
decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state) 

decoder_instance = tfa.seq2seq.BeamSearchDecoder(decoderNetwork.rnn_cell,beam_width=beam_width,
                                                 output_layer=decoderNetwork.dense_layer)


# Since we do not know the target sequence lengths in advance, we use maximum_iterations to limit the translation lengths.
# One heuristic is to decode up to two times the source sentence lengths.
maximum_iterations = tf.round(tf.reduce_max(Tx) * 2)

#initialize inference decoder

(first_finished, first_inputs,first_state) = decoder_instance.initialize(decoder_embedding_matrix,
                             start_tokens = start_tokens,
                             end_token=end_token,
                             initial_state = decoder_initial_state)
#print( first_finished.shape)
print("\nfirst_inputs returns the same decoder_input i.e. embedding of  <start> :",first_inputs.shape)

inputs = first_inputs
state = first_state  
predictions = np.empty((inference_batch_size, beam_width,0), dtype = np.int32)
beam_scores =  np.empty((inference_batch_size, beam_width,0), dtype = np.float32)                                                                            
for j in range(maximum_iterations):
    beam_search_outputs, next_state, next_inputs, finished = decoder_instance.step(j,inputs,state)
    inputs = next_inputs
    state = next_state
    outputs = np.expand_dims(beam_search_outputs.predicted_ids,axis = -1)
    scores = np.expand_dims(beam_search_outputs.scores,axis = -1)
    predictions = np.append(predictions, outputs, axis = -1)
    beam_scores = np.append(beam_scores, scores, axis = -1)
print(predictions.shape) 
print(beam_scores.shape)


beam_with * [batch_size, Tx, rnn_units] :  3 * [2, Tx, rnn_units]] : (18, 25, 1000)

first_inputs returns the same decoder_input i.e. embedding of  <start> : (6, 3, 50)
(6, 3, 50)
(6, 3, 50)


In [252]:
for i in range(len(predictions)):
    print("---------------------------------------------")
    output_beams_per_sample = predictions[i,:,:]
    score_beams_per_sample = beam_scores[i,:,:]
    for beam, score in zip(output_beams_per_sample,score_beams_per_sample) :
        seq = list(itertools.takewhile( lambda index: index !=2, beam))
        score_indexes = np.arange(len(seq))
        beam_score = score[score_indexes].sum()
        print(" ".join( [index_word[w] for w in seq]), " beam score: ", beam_score)


---------------------------------------------
i might to us <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>  beam score:  -144.82547
you , hes . to your <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>  beam score:  -215.70377
my might to a it . new . <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>  beam score:  -234.18427
----------------------------